# 汇率预测机器人

In [13]:
import numpy as np
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM
from keras.layers import LSTM, Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

class CurrencyLSTMModel:
    def __init__(self, currency_name, data_path, look_back=1):
        self.currency_name = currency_name
        self.data_path = data_path
        self.look_back = look_back
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None

    def load_and_prepare_data(self):
        dataset = read_csv(self.data_path, header=0, index_col=0)
        values = dataset.values.astype('float32')
        scaled = self.scaler.fit_transform(values)
        
        reframed = series_to_supervised(scaled, self.look_back, 1)
        values = reframed.values
        n_train = int(len(values) * 0.7)
        train = values[:n_train, :]
        test = values[n_train:, :]
        
        train_X, train_y = train[:, :-1], train[:, -1]
        test_X, test_y = test[:, :-1], test[:, -1]
        
        train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], 1))
        test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], 1))
        
        return train_X, train_y, test_X, test_y

    def build_model(self, input_shape):
        model = Sequential()
        model.add(LSTM(50, input_shape=input_shape, return_sequences=True, kernel_regularizer=l2(0.01)))
        model.add(Dropout(0.3))  # Dropout to prevent overfitting
        model.add(LSTM(100, return_sequences=True, kernel_regularizer=l2(0.01)))
        model.add(Dropout(0.3))  # Dropout to prevent overfitting
        model.add(LSTM(50, return_sequences=False, kernel_regularizer=l2(0.01)))
        model.add(Dropout(0.3))  # Dropout to prevent overfitting
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='adam')
        self.model = model

    def train(self, epochs=100, batch_size=60):
        train_X, train_y, test_X, test_y = self.load_and_prepare_data()
        self.build_model((train_X.shape[1], train_X.shape[2]))

        # Early stopping and learning rate scheduler
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

        # Train the model with early stopping and learning rate scheduling
        history = self.model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size,
                                 validation_data=(test_X, test_y), verbose=2, shuffle=False,
                                 callbacks=[early_stopping, lr_scheduler])
        return history

    def save(self, model_path):
        self.model.save(model_path)

    def load(self, model_path):
        self.model = load_model(model_path)

    def predict(self, X):
        yhat = self.model.predict(X)
        return yhat



## 新币汇率预测

In [6]:
import numpy as np
import pandas as pd
import joblib


def predict(name:str):
    #CNY_HKD_to_exchange_rate.csv
    model = CurrencyLSTMModel(currency_name=name, data_path=f"CNY_{name}_to_exchange_rate.csv", look_back=100)
    model.train()
    model.save(f"{name}_model.keras")
    model.load(f"{name}_model.keras")
    joblib.dump(model.scaler, 'scaler.save')
    model.scaler = joblib.load('scaler.save')
    csvname = f"CNY_{name}_to_exchange_rate.csv"
    # 读取原始数据
    dataset = pd.read_csv(csvname, header=0, index_col=0)
    values = dataset.values.astype('float32')

    # 归一化
    scaled = model.scaler.transform(values)

    # 取最近100天
    last_100 = scaled[-100:]  # shape: (100, 1)

    # 递归预测未来20天
    future_steps = 20
    future_preds = []

    input_seq = last_100.reshape((1, 100, 1))

    for _ in range(future_steps):
        yhat = model.predict(input_seq)
        future_preds.append(yhat[0, 0])
        # 更新输入序列：去掉最早一天，加上最新预测
        new_input = np.append(input_seq[0, 1:, 0], yhat[0, 0])
        input_seq = new_input.reshape((1, 100, 1))

    # 反归一化
    future_preds = np.array(future_preds).reshape(-1, 1)
    future_real = model.scaler.inverse_transform(future_preds)
    print("未来20天预测汇率：")
    print(future_real.flatten())
    

targets = ["JPY", "KRW", "SGD", "THB", "HKD", "MYR"]  # 多个目标货币
for target in targets:
    print(f"预测 {target} 对人民币的汇率：")
    predict(target)
    print("-" * 30)

预测 JPY 对人民币的汇率：
Epoch 1/100


/home/rprp/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 - 4s - 2s/step - loss: 0.2742 - val_loss: 0.1743
Epoch 2/100
2/2 - 1s - 293ms/step - loss: 0.1040 - val_loss: 0.4041
Epoch 3/100
2/2 - 0s - 240ms/step - loss: 0.2181 - val_loss: 0.2054
Epoch 4/100
2/2 - 0s - 236ms/step - loss: 0.1417 - val_loss: 0.1293
Epoch 5/100
2/2 - 0s - 247ms/step - loss: 0.1135 - val_loss: 0.1571
Epoch 6/100
2/2 - 1s - 257ms/step - loss: 0.1233 - val_loss: 0.1597
Epoch 7/100
2/2 - 0s - 242ms/step - loss: 0.1176 - val_loss: 0.1462
Epoch 8/100
2/2 - 1s - 281ms/step - loss: 0.1010 - val_loss: 0.1272
Epoch 9/100
2/2 - 0s - 245ms/step - loss: 0.0979 - val_loss: 0.1200
Epoch 10/100
2/2 - 0s - 245ms/step - loss: 0.1025 - val_loss: 0.1235
Epoch 11/100
2/2 - 0s - 228ms/step - loss: 0.1061 - val_loss: 0.1232
Epoch 12/100
2/2 - 0s - 225ms/step - loss: 0.1061 - val_loss: 0.1185
Epoch 13/100
2/2 - 0s - 235ms/step - loss: 0.1028 - val_loss: 0.1180
Epoch 14/100
2/2 - 0s - 227ms/step - loss: 0.0988 - val_loss: 0.1212
Epoch 15/100
2/2 - 0s - 226ms/step - loss: 0.0963 - val_lo

/home/rprp/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 - 5s - 3s/step - loss: 0.4409 - val_loss: 0.5771
Epoch 2/100
2/2 - 1s - 255ms/step - loss: 0.2970 - val_loss: 0.2600
Epoch 3/100
2/2 - 1s - 268ms/step - loss: 0.1113 - val_loss: 0.3804
Epoch 4/100
2/2 - 0s - 237ms/step - loss: 0.2285 - val_loss: 0.0965
Epoch 5/100
2/2 - 0s - 248ms/step - loss: 0.1398 - val_loss: 0.1505
Epoch 6/100
2/2 - 0s - 246ms/step - loss: 0.1518 - val_loss: 0.2046
Epoch 7/100
2/2 - 1s - 256ms/step - loss: 0.1523 - val_loss: 0.1758
Epoch 8/100
2/2 - 0s - 249ms/step - loss: 0.1310 - val_loss: 0.1017
Epoch 9/100
2/2 - 0s - 233ms/step - loss: 0.1099 - val_loss: 0.0736
Epoch 10/100
2/2 - 0s - 243ms/step - loss: 0.1180 - val_loss: 0.1278
Epoch 11/100
2/2 - 0s - 231ms/step - loss: 0.1290 - val_loss: 0.1007
Epoch 12/100
2/2 - 0s - 249ms/step - loss: 0.1180 - val_loss: 0.0661
Epoch 13/100
2/2 - 1s - 251ms/step - loss: 0.1096 - val_loss: 0.0710
Epoch 14/100
2/2 - 1s - 251ms/step - loss: 0.1069 - val_loss: 0.0763
Epoch 15/100
2/2 - 0s - 247ms/step - loss: 0.1047 - val_lo

...

2/2 - 5s - 2s/step - loss: 0.5515 - val_loss: 0.1082
Epoch 2/100
2/2 - 1s - 261ms/step - loss: 0.3031 - val_loss: 0.2260
Epoch 3/100
2/2 - 0s - 234ms/step - loss: 0.1381 - val_loss: 0.3816
Epoch 4/100
2/2 - 1s - 509ms/step - loss: 0.1559 - val_loss: 0.1700
Epoch 5/100
2/2 - 0s - 238ms/step - loss: 0.1089 - val_loss: 0.1190
Epoch 6/100
2/2 - 0s - 216ms/step - loss: 0.1429 - val_loss: 0.1369
Epoch 7/100
2/2 - 0s - 204ms/step - loss: 0.1061 - val_loss: 0.2093
Epoch 8/100
2/2 - 0s - 222ms/step - loss: 0.0739 - val_loss: 0.2889
Epoch 9/100
2/2 - 0s - 243ms/step - loss: 0.1015 - val_loss: 0.2748
Epoch 10/100
2/2 - 0s - 246ms/step - loss: 0.0840 - val_loss: 0.2139
Epoch 11/100
2/2 - 0s - 220ms/step - loss: 0.0750 - val_loss: 0.1775
Epoch 12/100
2/2 - 0s - 246ms/step - loss: 0.0858 - val_loss: 0.1777
Epoch 13/100
2/2 - 0s - 231ms/step - loss: 0.0820 - val_loss: 0.2055
Epoch 14/100
2/2 - 1s - 264ms/step - loss: 0.0705 - val_loss: 0.2451
Epoch 15/100
2/2 - 1s - 279ms/step - loss: 0.0753 - val_lo

/home/rprp/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 - 6s - 3s/step - loss: 0.4856 - val_loss: 0.2518
Epoch 2/100
2/2 - 1s - 274ms/step - loss: 0.2970 - val_loss: 0.1135
Epoch 3/100
2/2 - 1s - 290ms/step - loss: 0.2126 - val_loss: 0.0515
Epoch 4/100
2/2 - 0s - 243ms/step - loss: 0.1653 - val_loss: 0.0638
Epoch 5/100
2/2 - 0s - 243ms/step - loss: 0.1295 - val_loss: 0.1059
Epoch 6/100
2/2 - 0s - 236ms/step - loss: 0.1474 - val_loss: 0.1217
Epoch 7/100
2/2 - 0s - 232ms/step - loss: 0.1585 - val_loss: 0.1171
Epoch 8/100
2/2 - 1s - 277ms/step - loss: 0.1541 - val_loss: 0.0974
Epoch 9/100
2/2 - 1s - 258ms/step - loss: 0.1395 - val_loss: 0.0698
Epoch 10/100
2/2 - 0s - 238ms/step - loss: 0.1285 - val_loss: 0.0661
Epoch 11/100
2/2 - 1s - 265ms/step - loss: 0.1213 - val_loss: 0.0839
Epoch 12/100
2/2 - 0s - 249ms/step - loss: 0.1088 - val_loss: 0.1126
Epoch 13/100
2/2 - 1s - 251ms/step - loss: 0.1174 - val_loss: 0.1203
Epoch 14/100
2/2 - 1s - 508ms/step - loss: 0.1212 - val_loss: 0.1107
Epoch 15/100
2/2 - 0s - 239ms/step - loss: 0.1116 - val_lo

/home/rprp/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 - 4s - 2s/step - loss: 0.4988 - val_loss: 0.1896
Epoch 2/100
2/2 - 1s - 281ms/step - loss: 0.3284 - val_loss: 0.1393
Epoch 3/100
2/2 - 1s - 290ms/step - loss: 0.1057 - val_loss: 0.4369
Epoch 4/100
2/2 - 0s - 245ms/step - loss: 0.1862 - val_loss: 0.1952
Epoch 5/100
2/2 - 0s - 232ms/step - loss: 0.0941 - val_loss: 0.1262
Epoch 6/100
2/2 - 0s - 229ms/step - loss: 0.1375 - val_loss: 0.1278
Epoch 7/100
2/2 - 0s - 203ms/step - loss: 0.1098 - val_loss: 0.1654
Epoch 8/100
2/2 - 0s - 190ms/step - loss: 0.0603 - val_loss: 0.2542
Epoch 9/100
2/2 - 0s - 195ms/step - loss: 0.0909 - val_loss: 0.2697
Epoch 10/100
2/2 - 0s - 247ms/step - loss: 0.0983 - val_loss: 0.2251
Epoch 11/100
2/2 - 1s - 282ms/step - loss: 0.0791 - val_loss: 0.1685
Epoch 12/100
2/2 - 1s - 275ms/step - loss: 0.0726 - val_loss: 0.1446
Epoch 13/100
2/2 - 0s - 242ms/step - loss: 0.0777 - val_loss: 0.1488
Epoch 14/100
2/2 - 0s - 247ms/step - loss: 0.0698 - val_loss: 0.1736
Epoch 15/100
2/2 - 0s - 237ms/step - loss: 0.0633 - val_lo

In [14]:
predict("SGD")

Epoch 1/100


/home/rprp/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 - 5s - 3s/step - loss: 2.4845 - val_loss: 2.7295 - learning_rate: 1.0000e-03
Epoch 2/100
2/2 - 1s - 279ms/step - loss: 2.2908 - val_loss: 2.3332 - learning_rate: 1.0000e-03
Epoch 3/100
2/2 - 0s - 243ms/step - loss: 2.1108 - val_loss: 2.0194 - learning_rate: 1.0000e-03
Epoch 4/100
2/2 - 0s - 211ms/step - loss: 2.0972 - val_loss: 1.9979 - learning_rate: 1.0000e-03
Epoch 5/100
2/2 - 0s - 200ms/step - loss: 1.9804 - val_loss: 2.0662 - learning_rate: 1.0000e-03
Epoch 6/100
2/2 - 0s - 194ms/step - loss: 1.8952 - val_loss: 2.0555 - learning_rate: 1.0000e-03
Epoch 7/100
2/2 - 0s - 190ms/step - loss: 1.8131 - val_loss: 1.9643 - learning_rate: 1.0000e-03
Epoch 8/100
2/2 - 0s - 202ms/step - loss: 1.7286 - val_loss: 1.8324 - learning_rate: 1.0000e-03
Epoch 9/100
2/2 - 0s - 202ms/step - loss: 1.6483 - val_loss: 1.6886 - learning_rate: 1.0000e-03
Epoch 10/100
2/2 - 0s - 209ms/step - loss: 1.5892 - val_loss: 1.5777 - learning_rate: 1.0000e-03
Epoch 11/100
2/2 - 0s - 246ms/step - loss: 1.5177 - va